<a href="https://colab.research.google.com/github/caileymm/cuda-by-example-exercises/blob/main/chapter_5_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.11.13
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp57msb0sa".


In [ ]:
%%writefile book.h

#ifndef __BOOK_H__
#define __BOOK_H__
#include <stdio.h>

static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))


#define HANDLE_NULL( a ) {if (a == NULL) { \
                            printf( "Host memory failed in %s at line %d\n", \
                                    __FILE__, __LINE__ ); \
                            exit( EXIT_FAILURE );}}

template< typename T >
void swap( T& a, T& b ) {
    T t = a;
    a = b;
    b = t;
}


void* big_random_block( int size ) {
    unsigned char *data = (unsigned char*)malloc( size );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}

int* big_random_block_int( int size ) {
    int *data = (int*)malloc( size * sizeof(int) );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}


// a place for common kernels - starts here

__device__ unsigned char value( float n1, float n2, int hue ) {
    if (hue > 360)      hue -= 360;
    else if (hue < 0)   hue += 360;

    if (hue < 60)
        return (unsigned char)(255 * (n1 + (n2-n1)*hue/60));
    if (hue < 180)
        return (unsigned char)(255 * n2);
    if (hue < 240)
        return (unsigned char)(255 * (n1 + (n2-n1)*(240-hue)/60));
    return (unsigned char)(255 * n1);
}

__global__ void float_to_color( unsigned char *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset*4 + 0] = value( m1, m2, h+120 );
    optr[offset*4 + 1] = value( m1, m2, h );
    optr[offset*4 + 2] = value( m1, m2, h -120 );
    optr[offset*4 + 3] = 255;
}

__global__ void float_to_color( uchar4 *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset].x = value( m1, m2, h+120 );
    optr[offset].y = value( m1, m2, h );
    optr[offset].z = value( m1, m2, h -120 );
    optr[offset].w = 255;
}


#if _WIN32
    //Windows threads.
    #include <windows.h>

    typedef HANDLE CUTThread;
    typedef unsigned (WINAPI *CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC unsigned WINAPI
    #define  CUT_THREADEND return 0

#else
    //POSIX threads.
    #include <pthread.h>

    typedef pthread_t CUTThread;
    typedef void *(*CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC void
    #define  CUT_THREADEND
#endif

//Create thread.
CUTThread start_thread( CUT_THREADROUTINE, void *data );

//Wait for thread to finish.
void end_thread( CUTThread thread );

//Destroy thread.
void destroy_thread( CUTThread thread );

//Wait for multiple threads.
void wait_for_threads( const CUTThread *threads, int num );

#if _WIN32
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void *data){
        return CreateThread(NULL, 0, (LPTHREAD_START_ROUTINE)func, data, 0, NULL);
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        WaitForSingleObject(thread, INFINITE);
        CloseHandle(thread);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        TerminateThread(thread, 0);
        CloseHandle(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        WaitForMultipleObjects(num, threads, true, INFINITE);

        for(int i = 0; i < num; i++)
            CloseHandle(threads[i]);
    }

#else
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void * data){
        pthread_t thread;
        pthread_create(&thread, NULL, func, data);
        return thread;
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        pthread_join(thread, NULL);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        pthread_cancel(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        for(int i = 0; i < num; i++)
            end_thread( threads[i] );
    }

#endif
#endif  // __BOOK_H__

Overwriting book.h


In [ ]:
%%writefile vector_sums_1.cu

// 5.2.1 Vector Sums: Redux
// GPU Vector Sums with Threads

#include "book.h"

#define N 10

__global__ void add( int *a, int *b, int *c ) {
  int tid = threadIdx.x; // index the data by thread index
  if (tid < N)
    c[tid] = a[tid] + b[tid];
}

int main( void ) {
  int a[N], b[N], c[N];
  int *dev_a, *dev_b, *dev_c;

  // allocate the memory on the GPU
  HANDLE_ERROR(cudaMalloc((void**)&dev_a, N * sizeof(int)));
  HANDLE_ERROR(cudaMalloc((void**)&dev_b, N * sizeof(int)));
  HANDLE_ERROR(cudaMalloc((void**)&dev_c, N * sizeof(int)));

  // fill the arrays ‘a’ and ‘b’ on the CPU
  for (int i = 0; i < N; i++) {
    a[i] = i;
    b[i] = i * i;
  }

  // copy the arrays ‘a’ and ‘b’ to the GPU
  HANDLE_ERROR(cudaMemcpy(dev_a, a, N * sizeof(int), cudaMemcpyHostToDevice));
  HANDLE_ERROR(cudaMemcpy(dev_b, b, N * sizeof(int), cudaMemcpyHostToDevice));
  add<<<1,N>>>(dev_a, dev_b, dev_c); // N threads in one block

  // copy the array ‘c’ back from the GPU to the CPU
  HANDLE_ERROR(cudaMemcpy(c, dev_c, N * sizeof(int), cudaMemcpyDeviceToHost));

  // display the results
  for (int i = 0; i < N; i++) {
    printf("%d + %d = %d\n", a[i], b[i], c[i]);
  }

  // free the memory allocated on the GPU
  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);

  return 0;
}

// NOTES:
// - Blocks: Groups of threads that execute a kernel in parallel.
// - Each block can handle a portion of the data, allowing for efficient
// - parallel processing across the GPU.
// - <kernel function name><<<N,M>>>(): M is the number of threads per block
// - N blocks * M threads/block

Writing vector_sums_1.cu


In [ ]:
!nvcc vector_sums_1.cu -o vector_sums_1
!./vector_sums_1

0 + 0 = 0
1 + 1 = 0
2 + 4 = 0
3 + 9 = 0
4 + 16 = 0
5 + 25 = 0
6 + 36 = 0
7 + 49 = 0
8 + 64 = 0
9 + 81 = 0


In [ ]:
%%writefile vector_sums_2.cu

// 5.2.1 Vector Sums: Redux
// GPU Sums OF Arbitrarily Long Vectors

# include "book.h"
# define N 33 * 1024

__global__ void add(int *a, int *b, int *c) {
  // calculates id of thread across all blocks in the grid
  int tid = threadIdx.x + blockIdx.x * blockDim.x;
  // grid-stride loop
  while (tid < N) {
    c[tid] = a[tid] + b[tid];
    // add total number of threads in the grid to the current tid
    tid += blockDim.x * gridDim.x;
  }
}

int main( void ) {
  int a[N], b[N], c[N];
  int *dev_a, *dev_b, *dev_c;

  // allocate the memory on the GPU
  HANDLE_ERROR(cudaMalloc((void**)&dev_a, N * sizeof(int)));
  HANDLE_ERROR(cudaMalloc((void**)&dev_b, N * sizeof(int)));
  HANDLE_ERROR(cudaMalloc((void**)&dev_c, N * sizeof(int)));

  // fill the arrays ‘a’ and ‘b’ on the CPU
  for (int i = 0; i < N; i++) {
    a[i] = i;
    b[i] = i * i;
  }

  // copy the arrays 'a' and 'b' to the GPU
  HANDLE_ERROR(cudaMemcpy(dev_a, a, N * sizeof(int), cudaMemcpyHostToDevice));
  HANDLE_ERROR(cudaMemcpy(dev_b, b, N * sizeof(int), cudaMemcpyHostToDevice));
  add<<<128,128>>>(dev_a, dev_b, dev_c); // 128 * 128 = 16384 total threads

  // copy the array 'c' back from the GPU to the CPU
  HANDLE_ERROR(cudaMemcpy(c, dev_c, N * sizeof(int), cudaMemcpyDeviceToHost));

  // verify that the GPU did the work we requested
  bool success = true;
  for (int i = 0; i < N; i++) {
    if ((a[i] + b[i]) != c[i]) {
      // printf("Error: %d + %d != %d\n", a[i], b[i], c[i]);
      success = false;
    }
  }

  if (success) printf("We did it!\n");

  // free the memory allocated on the GPU
  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);
  return 0;

  // NOTES:
  // - threadIdx.x: Thread index within its block in the x-dimension.
  // - blockIdx.x: Block index within the grid in the x-dimension.
  // - blockDim.x: Number of threads per block in the x-dimension.
  // - Grid-stride loop ensures that all elements in the array are processed,
  // - even if the number of elements N is much larger than the total number of
  // - threads launched.
  // - 16384 total threads but N = 33792, so grid-stride loop is necessary

}

Overwriting vector_sums_2.cu


In [ ]:
!nvcc vector_sums_2.cu -o vector_sums_2
!./vector_sums_2

In [ ]:
%%writefile dot_product.cu

// 5.3.1 Dot Product

#include "book.h"

#define imin(a, b) (a < b ? a : b)

const int N = 33 * 1024; // size of vectors
const int threadsPerBlock = 256;
const int blocksPerGrid = imin(32, (N + threadsPerBlock - 1) / threadsPerBlock);

__global__ void dot(float *a, float *b, float *c) {
  __shared__ float cache[threadsPerBlock]; // shared memory per BLOCK
  int tid = threadIdx.x + blockIdx.x * blockDim.x; // global thread index
  int cacheIndex = threadIdx.x; // local thread index
  float temp = 0;

  // grid-stride loop
  // each thread calculates a sum of products (SUBSET of total products)
  while (tid < N) {
    temp += a[tid] * b[tid];
    tid += blockDim.x * gridDim.x;
  }

  cache[cacheIndex] = temp; // stores each thread's sum of products

  __syncthreads(); // ensures all threads in BLOCK calculated sum of products

  // parallel reduction (executed once per block)
  // sum of each thread's sum of products
  int i = blockDim.x / 2; // half of threads per block
  while (i != 0) {
    // threads with index below i sum with threads with index above i
    if (cacheIndex < i)
      cache[cacheIndex] += cache[cacheIndex + i];
    __syncthreads(); // ensures all threads within block have completed the addtion in the current iteration
    i /= 2;
  }

  // first thread in block holds sum of all sum of products
  if (cacheIndex == 0)
    c[blockIdx.x] = cache[0];
}

int main(void) {
  float *a, *b, c, *partial_c;
  float *dev_a, *dev_b, *dev_partial_c;

  // allocate memory on the CPU side
  a = (float*) malloc(N * sizeof(float));
  b = (float*) malloc(N * sizeof(float));
  partial_c = (float*) malloc(blocksPerGrid * sizeof(float));

  // allocate the memory on the GPU
  HANDLE_ERROR(cudaMalloc((void**) &dev_a, N * sizeof(float)));
  HANDLE_ERROR(cudaMalloc((void**) &dev_b, N * sizeof(float)));
  HANDLE_ERROR(cudaMalloc((void**) &dev_partial_c,blocksPerGrid * sizeof(float)));

  // fill in the host memory with data
  for (int i = 0; i < N; i++) {
    a[i] = i;
    b[i] = i * 2;
  }

  // copy the arrays ‘a’ and ‘b’ to the GPU
  HANDLE_ERROR(cudaMemcpy(dev_a, a, N * sizeof(float), cudaMemcpyHostToDevice));
  HANDLE_ERROR(cudaMemcpy(dev_b, b, N * sizeof(float), cudaMemcpyHostToDevice));

  dot<<<blocksPerGrid,threadsPerBlock>>>(dev_a, dev_b, dev_partial_c );

  // copy the array 'c' back from the GPU to the CPU
  HANDLE_ERROR(cudaMemcpy(partial_c, dev_partial_c, blocksPerGrid * sizeof(float), cudaMemcpyDeviceToHost));

  // finish up on the CPU side
  c = 0;
  for (int i = 0; i < blocksPerGrid; i++) {
    c += partial_c[i];
  }

  #define sum_squares(x) (x * (x + 1) * (2 * x + 1) / 6)

  printf("Does GPU value %.6g = %.6g?\n", c, 2 * sum_squares((float)(N - 1)));

  // free memory on the GPU side
  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_partial_c);

  // free memory on the CPU side
  free(a);
  free(b);
  free(partial_c);
}

// NOTES:
// - (N + threadsPerBlock - 1) / threadsPerBlock is the standard way to
//   calculate the number of blocks needed to process N elements, given that
//   each block can process threadsPerBlock elements
// - if (cacheIndex < i) {
//      cache[cacheIndex] += cache[cacheIndex + i];
//      __syncthreads();
//   }
//   This results in a deadlock! The threads that reaches __syncthreads()
//   (index < i) will wait indefinitely for threads that skipped it (index >= i)

Writing dot_product.cu


In [ ]:
!nvcc dot_product.cu -o dot_product
!./dot_product

Does GPU value 0 = 2.57236e+13?
